In [1]:
import os
import numpy as np
from collections import Counter
from sklearn import svm
from sklearn.metrics import accuracy_score

In [2]:
TRAIN_DIR = "./data/svm_pratice_phase2/train-mails"
TEST_DIR = "./data/svm_pratice_phase2/test-mails"


# Data cleaning + feature engineering

In [3]:
def make_dictionary(root_dir):
    #this function returns a frequency counter about the top 3000 commonly used words in emails
    all_words = []
    emails = [os.path.join(root_dir, f) for f in os.listdir(root_dir)]
    for mail in emails:
        with open(mail) as m:
            for line in m:
                words = line.split()
                all_words += words
    dictionary = Counter(all_words)
    
    list_to_remove = list(dictionary)
    #list_to_remove = dictionary.keys()
    
    for item in list_to_remove:
        if item.isalpha() == False:
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    
    dictionary = dictionary.most_common(3000)
    
    return dictionary

def extract_features(mail_dir):
    files = [os.path.join(mail_dir, fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files), 3000))
    train_labels = np.zeros(len(files))
    count = 0
    docID = 0
    for fil in files:
        with open(fil) as fi:
            for i, line in enumerate(fi):
                if i == 2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i, d in enumerate(dictionary):
                            if d[0] == word:
                                wordID = i
                                features_matrix[docID, wordID] = words.count(word)
        train_labels[docID] = 0
        filepathTokens = fil.split('/')
        lastToken = filepathTokens[len(filepathTokens) - 1]
        if(lastToken.startswith("spmsg")):
           train_labels[docID] = 1
           count = count + 1
        docID = docID + 1
    return features_matrix, train_labels

In [15]:

dictionary = make_dictionary(TRAIN_DIR)
print("reading and processing emails from files.")
feature_matrix, labels = extract_features(TRAIN_DIR)
test_feature_matrix, test_labels = extract_features(TEST_DIR)

reading and processing emails from files.


In [5]:
print(feature_matrix.shape)

(702, 3000)


# Building model

In [6]:
model = svm.SVC()

print("Training model.")

#train model
model.fit(feature_matrix, labels)

predicted_labels = model.predict(test_feature_matrix)

print("FINISHED classifying. accuracy score : ")
print(accuracy_score(test_labels, predicted_labels))

Training model.
FINISHED classifying. accuracy score : 
0.926923076923077


In [16]:
#removing 10% train and test data for testing purpose 

# feature_matrix = feature_matrix[:len(feature_matrix)/10]
# labels = labels[:len(labels)/10]

# model = svm.SVC()

# print("Training model.")

# #train model
# model.fit(feature_matrix, labels)

# predicted_labels = model.predict(test_feature_matrix)

# print("FINISHED classifying. accuracy score : ")
# print(accuracy_score(test_labels, predicted_labels))

TypeError: slice indices must be integers or None or have an __index__ method